In [1]:
import pandas as pd

In [2]:
pd.read_csv(r"C:\Users\SAMSUNG\Desktop\위경도데이터.csv")

In [6]:
data2=pd.read_csv("./침수흔적도.csv")

In [9]:
data2.columns

Index(['OBJECTID', 'GU_NAM', 'F_SHIM', 'F_DISA_NM', 'F_AVR_HGT', 'F_AREA',
       'ADM_CD', 'F_SAT_YMD', 'F_END_YMD', 'F_SAT_TM', 'F_END_TM', 'F_RSN_DTL',
       'F_ZONE_NM', 'INV_YR', 'F_YR', 'TYPE', 'OBJECTID_1', 'OBJECTID_2',
       'BASE_YEAR', 'ADM_CD_2', 'ADM_NM', 'Shape_Area', 'GU_CODE', 'GU_NAME',
       'POP'],
      dtype='object')

In [20]:
data["주소"]=data["주소"].apply(lambda x:x.replace("서울","서울특별시"))

In [22]:
temp_result=pd.merge(data2,data,left_on="F_ZONE_NM",right_on="주소",how="left")

In [24]:
temp_result.to_csv("임시_위경도_데이터.csv")

추가 위경도..

In [30]:
need_add=list(temp_result[temp_result["경도"].isna()]["F_ZONE_NM"])

In [36]:
len(need_add)

4736

In [38]:
import json
import requests as rq
def getAddr(addrs):
    REST_API_KEY='19111005ad4c10663591e0a55b344ba5' #'4fd65a1fc1d9de84fb9809d081f3fc8c'
    headers = {"Authorization": f"KakaoAK {REST_API_KEY}"}
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+addrs+"&analyze_type=exact"
    model=rq.get(url,headers=headers)
    if model.status_code!=200:
        return addrs
    else:
        return json.loads(str(rq.get(url,headers=headers).text))['documents']

In [39]:
import tqdm
pd.DataFrame(columns=["경도","위도","주소"]).to_csv("./위경도_추가데이터.csv")

In [40]:
def makexy(i,j):
    with tqdm.tqdm(total=j-i,unit="B",unit_scale=True) as process_bar:
        for addrs in need_add[i:j]:
            temp=getAddr(addrs)
            try:
                xy_data=[{'경도':temp[0]["x"],"위도":temp[0]["x"],'주소':temp[0]['address_name']}]
                merge_data=pd.DataFrame(xy_data)
                merged_data=pd.read_csv("./위경도_추가데이터.csv")
                add_xy_data=pd.concat([merged_data,merge_data])
                add_xy_data[["경도","위도","주소"]].to_csv("./위경도_추가데이터.csv")
            except:
                continue
            process_bar.update(1)

In [46]:
makexy(257,1000)

 55%|█████▌    | 411/743 [25:58<20:58, 3.79s/B]   


KeyboardInterrupt: 

In [47]:
pd.read_csv("./위경도_추가데이터.csv")

,Unnamed: 0,경도,위도,주소
0,0,127.006001,127.006001,서울 강북구 우이동 산 8-15
1,1,126.909160,126.909160,서울 영등포구 대림동 951-3
2,2,126.908853,126.908853,서울 영등포구 대림동 952-7
3,3,126.909244,126.909244,서울 영등포구 대림동 951-4
4,4,126.909244,126.909244,서울 영등포구 대림동 951-4
...,...,...,...,...
663,663,126.909057,126.909057,서울 영등포구 신길동 4735
664,664,126.909301,126.909301,서울 영등포구 신길동 4715
665,665,126.909306,126.909306,서울 영등포구 대림동 951-1
666,666,126.908957,126.908957,서울 영등포구 대림동 952-3
